In [1]:
import random
from mesa import Agent, Model
from mesa.time import BaseScheduler
from mesa.space import Grid,MultiGrid
from mesa.datacollection import DataCollector
from mesa.visualization.modules import CanvasGrid
from mesa.visualization.ModularVisualization import ModularServer

In [2]:
NUMBER_STATIONS = 25
BATTERY_SIZE = 900
class Drone(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.pos = model.starting_pos
        self.battery = BATTERY_SIZE
        self.Type = 'Drone'
        self.currentCapacity = 0
        self.stationVisited = 0
    def step(self):
        print("Batería Actual:", self.battery)
        in_station = False
        if(len(self.model.grid[self.pos]) > 1):
            base = True
            station_obj = None
            
            for e in self.model.grid[self.pos]:
                if e.Type == "Station":
                    station_obj = e
                    base = False
            if(base):
                print("dron cargado")
                self.battery = BATTERY_SIZE
                if self.stationVisited == NUMBER_STATIONS:
                    self.model.schedule.remove(self)
                    return
            else:
                if station_obj.dataSize > 10:
                    print(f"descargando -> capacidad del drone: {self.currentCapacity} kbps")
                    print(f"data en estación {station_obj.dataSize}")
                    self.currentCapacity = self.currentCapacity  + 10
                    station_obj.dataSize = station_obj.dataSize - 10
                    self.battery -= 1
                    in_station = True
                    
                elif station_obj.dataSize > 0:
                    print(f"descargando -> capacidad del drone: {self.currentCapacity} kbps")
                    print(f"data en estación {station_obj.dataSize}")
                    
                    
                    self.currentCapacity = self.currentCapacity + station_obj.dataSize
                    station_obj.dataSize = 0
                    self.stationVisited = self.stationVisited + 1
                    self.battery -= 1
                    station_obj.visit = True
                    in_station = False
                else:
                    in_station = False
                
                
        if self.battery > 0 and not in_station:
            self.move()
            self.battery -= 1
        elif self.battery > 0 and in_station:
            pass
        else:
            self.model.schedule.remove(self)
    
    def move(self):
        distances = []
        base_station_pos = (self.model.width//2, self.model.height//2) 
        for element in self.model.grid.coord_iter():
            agent, posx, posy = element
            if len(agent) >0 and agent[0].Type == "Station" and agent[0].visit == False:
                
                distance = abs(self.pos[0] - posx) + abs(self.pos[1] - posy)
                data = (distance, agent, (posx,posy))
                distances.append(data)
        
        
        if len(distances) == 0:
            distances.append((0, None, base_station_pos))
            
        distances.sort(key = lambda x: x[0])
        goal_posx = distances[0][2][0]
        goal_posy = distances[0][2][1]
        
        distance_to_base = abs(goal_posx - base_station_pos[0]) + abs(goal_posy - base_station_pos[1])
        
        if self.battery < distances[0][0] + distance_to_base: 
            #move to base
            goal_posx = base_station_pos[0]
            goal_posy = base_station_pos[1]
            if self.pos[0] !=  goal_posx:
                # + a la izq, - a la der 
                if self.pos[0] - goal_posx < 0:
                    new_position = (self.pos[0] +1 , self.pos[1])
                else:
                    new_position = (self.pos[0] -1 , self.pos[1])  
            else:
                # + a abajo, - a arriba 
                if self.pos[1] - goal_posy < 0:
                    new_position = (self.pos[0], self.pos[1] +1)
                else:    
                    new_position = (self.pos[0], self.pos[1] -1)
            self.model.grid.move_agent(self, new_position)
            return
        
        
        if self.pos[0] !=  goal_posx:
            # + a la izq, - a la der 
            if self.pos[0] - goal_posx < 0:
                new_position = (self.pos[0] +1 , self.pos[1])
            else:
                new_position = (self.pos[0] -1 , self.pos[1])  
        else:
            # + a abajo, - a arriba 
            if self.pos[1] - goal_posy < 0:
                new_position = (self.pos[0], self.pos[1] +1)
            else:    
                new_position = (self.pos[0], self.pos[1] -1)
        
        
        self.model.grid.move_agent(self, new_position)

class Station(Agent):
    def __init__(self, unique_id, model, dataSize):
        super().__init__(unique_id, model)
        self.pos = self.generate_valid_pos(model)
        #self.possible_points = random.sample([(x,y) for x in range(model.width) for y in range(model.height)], 3)
        self.dataSize = dataSize
        self.Type = 'Station'
        self.visit = False
        
    def generate_valid_pos(self, model):
        pos = random.sample([(x,y) for x in range(model.width) for y in range(model.height)], 1)[0]
        while pos in model.occupied_positions:
            pos = random.sample([(x,y) for x in range(model.width) for y in range(model.height)], 1)[0]
        model.occupied_positions.add(pos)
        return pos

class BaseStation(Agent):
    def __init__(self, unique_id, model, pos):
        super().__init__(unique_id, model)
        self.pos = pos
        self.possible_points = random.sample([(x,y) for x in range(model.width) for y in range(model.height)], 1)
        self.Type = 'BaseStation'
        model.occupied_positions.add(pos)
        
        
class DroneModel(Model):
    def __init__(self, N, width, height):
        self.width = width
        self.height = height
        self.num_drones = N
        self.grid = MultiGrid(width, height, torus=False)
        self.schedule = BaseScheduler(self)
        self.starting_pos = (width // 2, height // 2)
        self.occupied_positions = set()
        
        for i in range(self.num_drones):
            d = Drone(i, self)
            self.schedule.add(d)
            self.grid.place_agent(d, self.starting_pos)
            
            s = BaseStation(random.randrange(100000), self, self.starting_pos)
            self.schedule.add(s)
            self.grid.place_agent(s, self.starting_pos)
        
        for i in range(NUMBER_STATIONS):
            #pos = random.sample([(x,y) for x in range(width) for y in range(height)], 1)[0]
            s = Station(random.randrange(100000), self, random.randrange(1024))
            self.schedule.add(s)
            self.grid.place_agent(s, s.pos)
        
        #self.datacollector = DataCollector(
        #    agent_reporters={"Battery": lambda x: x.battery}
        #)

    def step(self):
        print(self.schedule)
        #self.datacollector.collect(self)
        self.schedule.step()
def drone_portrayal(agent):
    portrayal = {}
    if agent.Type == "Drone":
        portrayal["Layer"] = 1
        portrayal["Shape"]="circle"
        portrayal["Filled"] = "true"
        portrayal["r"] = 0.5
        if agent.battery > 0:
            portrayal["Color"] = "green"
        else:
            portrayal["Color"] = "red"
    if agent.Type == "Station":
        portrayal["Layer"] = 0
        portrayal["Shape"]="rect"
        portrayal["Filled"] = "true"
        portrayal["w"] = 1
        portrayal["h"] = 1
        if agent.visit:
            portrayal["Color"] = "black"
        else:
            portrayal["Color"] = "gray"
        
    if agent.Type == "BaseStation":
        portrayal["Layer"] = 0
        portrayal["Shape"]="rect"
        portrayal["Filled"] = "true"
        portrayal["w"] = 1
        portrayal["h"] = 1
        portrayal["Color"] = "blue"
    return portrayal
   


In [3]:
grid = CanvasGrid(drone_portrayal, 60, 60, 500, 500)
server = ModularServer(DroneModel, [grid], "Drone Model", {"N": 1, "width": 60, "height": 60})
server.port = 8510
server.launch()

Interface starting at http://127.0.0.1:8510
Opening in existing browser session.


RuntimeError: This event loop is already running

Socket opened!
{"type":"reset"}
{"type":"get_step","step":1}
Batería Actual: 900
dron cargado
{"type":"get_step","step":2}
Batería Actual: 899
{"type":"get_step","step":3}
Batería Actual: 898
{"type":"get_step","step":4}
Batería Actual: 897
{"type":"get_step","step":5}
Batería Actual: 896
{"type":"get_step","step":6}
Batería Actual: 895
{"type":"get_step","step":7}
Batería Actual: 894
{"type":"get_step","step":8}
Batería Actual: 893
{"type":"get_step","step":9}
Batería Actual: 892
{"type":"get_step","step":10}
Batería Actual: 891
{"type":"get_step","step":11}
Batería Actual: 890
descargando -> capacidad del drone: 0 kbps
data en estación 982
{"type":"get_step","step":12}
Batería Actual: 889
descargando -> capacidad del drone: 10 kbps
data en estación 972
{"type":"get_step","step":13}
Batería Actual: 888
descargando -> capacidad del drone: 20 kbps
data en estación 962
{"type":"get_step","step":14}
Batería Actual: 887
descargando -> capacidad del drone: 30 kbps
data en estación 952
{"typ

{"type":"get_step","step":55}
Batería Actual: 846
descargando -> capacidad del drone: 440 kbps
data en estación 542
{"type":"get_step","step":56}
Batería Actual: 845
descargando -> capacidad del drone: 450 kbps
data en estación 532
{"type":"get_step","step":57}
Batería Actual: 844
descargando -> capacidad del drone: 460 kbps
data en estación 522
{"type":"get_step","step":58}
Batería Actual: 843
descargando -> capacidad del drone: 470 kbps
data en estación 512
{"type":"get_step","step":59}
Batería Actual: 842
descargando -> capacidad del drone: 480 kbps
data en estación 502
{"type":"get_step","step":60}
Batería Actual: 841
descargando -> capacidad del drone: 490 kbps
data en estación 492
{"type":"get_step","step":61}
Batería Actual: 840
descargando -> capacidad del drone: 500 kbps
data en estación 482
{"type":"get_step","step":62}
Batería Actual: 839
descargando -> capacidad del drone: 510 kbps
data en estación 472
{"type":"get_step","step":63}
Batería Actual: 838
descargando -> capacid

{"type":"get_step","step":106}
Batería Actual: 795
descargando -> capacidad del drone: 950 kbps
data en estación 32
{"type":"get_step","step":107}
Batería Actual: 794
descargando -> capacidad del drone: 960 kbps
data en estación 22
{"type":"get_step","step":108}
Batería Actual: 793
descargando -> capacidad del drone: 970 kbps
data en estación 12
{"type":"get_step","step":109}
Batería Actual: 792
descargando -> capacidad del drone: 980 kbps
data en estación 2
{"type":"get_step","step":110}
Batería Actual: 790
{"type":"get_step","step":111}
Batería Actual: 789
{"type":"get_step","step":112}
Batería Actual: 788
{"type":"get_step","step":113}
Batería Actual: 787
{"type":"get_step","step":114}
Batería Actual: 786
{"type":"get_step","step":115}
Batería Actual: 785
{"type":"get_step","step":116}
Batería Actual: 784
descargando -> capacidad del drone: 982 kbps
data en estación 164
{"type":"get_step","step":117}
Batería Actual: 783
descargando -> capacidad del drone: 992 kbps
data en estación 1

...

{"type":"get_step","step":475}
Batería Actual: 418
descargando -> capacidad del drone: 4127 kbps
data en estación 323
{"type":"get_step","step":476}
Batería Actual: 417
descargando -> capacidad del drone: 4137 kbps
data en estación 313
{"type":"get_step","step":477}
Batería Actual: 416
descargando -> capacidad del drone: 4147 kbps
data en estación 303
{"type":"get_step","step":478}
Batería Actual: 415
descargando -> capacidad del drone: 4157 kbps
data en estación 293
{"type":"get_step","step":479}
Batería Actual: 414
descargando -> capacidad del drone: 4167 kbps
data en estación 283
{"type":"get_step","step":480}
Batería Actual: 413
descargando -> capacidad del drone: 4177 kbps
data en estación 273
{"type":"get_step","step":481}
Batería Actual: 412
descargando -> capacidad del drone: 4187 kbps
data en estación 263
{"type":"get_step","step":482}
Batería Actual: 411
descargando -> capacidad del drone: 4197 kbps
data en estación 253
{"type":"get_step","step":483}
Batería Actual: 410
desca

{"type":"get_step","step":528}
Batería Actual: 364
descargando -> capacidad del drone: 4600 kbps
data en estación 590
{"type":"get_step","step":529}
Batería Actual: 363
descargando -> capacidad del drone: 4610 kbps
data en estación 580
{"type":"get_step","step":530}
Batería Actual: 362
descargando -> capacidad del drone: 4620 kbps
data en estación 570
{"type":"get_step","step":531}
Batería Actual: 361
descargando -> capacidad del drone: 4630 kbps
data en estación 560
{"type":"get_step","step":532}
Batería Actual: 360
descargando -> capacidad del drone: 4640 kbps
data en estación 550
{"type":"get_step","step":533}
Batería Actual: 359
descargando -> capacidad del drone: 4650 kbps
data en estación 540
{"type":"get_step","step":534}
Batería Actual: 358
descargando -> capacidad del drone: 4660 kbps
data en estación 530
{"type":"get_step","step":535}
Batería Actual: 357
descargando -> capacidad del drone: 4670 kbps
data en estación 520
{"type":"get_step","step":536}
Batería Actual: 356
desca

{"type":"get_step","step":579}
Batería Actual: 313
descargando -> capacidad del drone: 5110 kbps
data en estación 80
{"type":"get_step","step":580}
Batería Actual: 312
descargando -> capacidad del drone: 5120 kbps
data en estación 70
{"type":"get_step","step":581}
Batería Actual: 311
descargando -> capacidad del drone: 5130 kbps
data en estación 60
{"type":"get_step","step":582}
Batería Actual: 310
descargando -> capacidad del drone: 5140 kbps
data en estación 50
{"type":"get_step","step":583}
Batería Actual: 309
descargando -> capacidad del drone: 5150 kbps
data en estación 40
{"type":"get_step","step":584}
Batería Actual: 308
descargando -> capacidad del drone: 5160 kbps
data en estación 30
{"type":"get_step","step":585}
Batería Actual: 307
descargando -> capacidad del drone: 5170 kbps
data en estación 20
{"type":"get_step","step":586}
Batería Actual: 306
descargando -> capacidad del drone: 5180 kbps
data en estación 10
{"type":"get_step","step":587}
Batería Actual: 304
{"type":"get_

{"type":"get_step","step":638}
Batería Actual: 251
descargando -> capacidad del drone: 5445 kbps
data en estación 1019
{"type":"get_step","step":639}
Batería Actual: 250
descargando -> capacidad del drone: 5455 kbps
data en estación 1009
{"type":"get_step","step":640}
Batería Actual: 249
descargando -> capacidad del drone: 5465 kbps
data en estación 999
{"type":"get_step","step":641}
Batería Actual: 248
descargando -> capacidad del drone: 5475 kbps
data en estación 989
{"type":"get_step","step":642}
Batería Actual: 247
descargando -> capacidad del drone: 5485 kbps
data en estación 979
{"type":"get_step","step":643}
Batería Actual: 246
descargando -> capacidad del drone: 5495 kbps
data en estación 969
{"type":"get_step","step":644}
Batería Actual: 245
descargando -> capacidad del drone: 5505 kbps
data en estación 959
{"type":"get_step","step":645}
Batería Actual: 244
descargando -> capacidad del drone: 5515 kbps
data en estación 949
{"type":"get_step","step":646}
Batería Actual: 243
des

{"type":"get_step","step":689}
Batería Actual: 200
descargando -> capacidad del drone: 5955 kbps
data en estación 509
{"type":"get_step","step":690}
Batería Actual: 199
descargando -> capacidad del drone: 5965 kbps
data en estación 499
{"type":"get_step","step":691}
Batería Actual: 198
descargando -> capacidad del drone: 5975 kbps
data en estación 489
{"type":"get_step","step":692}
Batería Actual: 197
descargando -> capacidad del drone: 5985 kbps
data en estación 479
{"type":"get_step","step":693}
Batería Actual: 196
descargando -> capacidad del drone: 5995 kbps
data en estación 469
{"type":"get_step","step":694}
Batería Actual: 195
descargando -> capacidad del drone: 6005 kbps
data en estación 459
{"type":"get_step","step":695}
Batería Actual: 194
descargando -> capacidad del drone: 6015 kbps
data en estación 449
{"type":"get_step","step":696}
Batería Actual: 193
descargando -> capacidad del drone: 6025 kbps
data en estación 439
{"type":"get_step","step":697}
Batería Actual: 192
desca

In [17]:
DroneModel1 = DroneModel(1,20,20)

In [18]:
DroneModel1.step()

Batería Actual: 900
dron cargado
[(17, [<__main__.Station object at 0x7fad0c86fca0>], (2, 19)), (5, [<__main__.Station object at 0x7fad0c86fbe0>], (14, 9)), (5, [<__main__.Station object at 0x7fad0c86f640>], (14, 11)), (17, [<__main__.Station object at 0x7fad0c86f5b0>], (17, 0)), (9, [<__main__.Station object at 0x7fad0c86fb20>], (19, 10))]
[(5, [<__main__.Station object at 0x7fad0c86fbe0>], (14, 9)), (5, [<__main__.Station object at 0x7fad0c86f640>], (14, 11)), (9, [<__main__.Station object at 0x7fad0c86fb20>], (19, 10)), (17, [<__main__.Station object at 0x7fad0c86fca0>], (2, 19)), (17, [<__main__.Station object at 0x7fad0c86f5b0>], (17, 0))]


In [216]:
server.reset_model()